In [2]:
import numpy as np
import requests
import re
import pandas as pd

This is how I get all of the pages for Komi Cant Communicate (First Step)

I have been reading Komi Can't Communicate and wanted to do some NLP and computer vision using the manga panel chapters \

base_url = 'https://komisanmanga.com/'

In [3]:
#Where the images to the website are hosted.
base_url = 'https://www.toonix.xyz/cdn_mangaraw/komi-san-wa-komyushou-desu/'
chapter_url = 'chapter-'

go_komi = requests.get(base_url)

In [4]:
chapters = re.findall('^.*chapter-.*$', go_komi.text, re.IGNORECASE | re.MULTILINE)

In [5]:
#creates a list of only the chapter
chap_number = [chapter.split('>')[0].split('-')[1].split('/')[0] for chapter in chapters if 'chapter-' in chapter]

We will put the list in a dataframe. Right now the chapters are organized incorrectly in the list. I will convert all of the chapter numbers into floats and then
use that information to organize them. I will make them floats so that the half chapters do not get lost.

In [6]:
chapters_frame = pd.DataFrame(chap_number, columns = ['chapter'])
chapters_frame.head()

,chapter
0,0
1,1
2,10
3,100
4,101


In [7]:
chapters_frame['float_chap'] = pd.to_numeric(chapters_frame['chapter'])

Now we want to sort our dataframe by this float_chap column

In [8]:
chapters_frame = chapters_frame.sort_values(by = 'float_chap')

It is now sorted by all of the chapters. The next step is to grab all of the images from each of the chapters

In [9]:
chapters_frame.head(15)

,chapter,float_chap
0,0,0.0
1,1,1.0
118,2,2.0
229,3,3.0
289,4,4.0
301,5,5.0
313,6,6.0
324,7,7.0
336,8,8.0
348,9,9.0


We need to recurvisely go through each of the chapters and figure out how many images each webpage has

In [11]:
for chapter in chapters_frame['chapter']:
    #create the link that we need to pull from the website

    chapter_link = base_url + chapter_url + chapter + '/'

    chapter_request = requests.get(chapter_link)
    jpg_lines =  re.findall('^.*.jpg.*$', chapter_request.text, re.IGNORECASE | re.MULTILINE)
    jpg_names = [jpg.split('>')[0].split('=')[1].split('"')[1] for jpg in jpg_lines]

    pictures_frame = pd.DataFrame(jpg_names, columns = ['jpg_num'])
    just_num = [num.split('.')[0] for num in pictures_frame['jpg_num']]
    pictures_frame['jpg_value'] = pd.to_numeric(just_num)
    pictures_frame = pictures_frame.sort_values(by = 'jpg_value')

    for jpg in pictures_frame['jpg_num']:
        full_url = chapter_link + jpg
        #gets the url with the image inside of it
        img_data = requests.get(full_url).content

        img_name = r'C:\Users\brunolopez\mldata\manga_scrape\komi\\' +  'chapter_' + str(chapter) + '_jpg_' + jpg
        with open(img_name, 'wb') as handler:
            handler.write(img_data)
    


Let's put everything we have done up above in a class so thjat I cana access it into the future.

In [ ]:
class MangaScrape:

    def __init__(self, base_url, chapter_url = ''):
        self.base_url = base_url
        self.chapter_url = chapter_url